In [199]:
# import pymongo
# clinet=pymongo.MongoClient("mongodb://10.120.27.23:27017/")
# db=clinet['project']
# collection = db.FinishTainanBigTable
# x=collection.find({})
# clinet.close()
# bigtable=x



In [201]:
# [dien['style'] for dien in bigtable]

[]

In [202]:
#經緯度換算公尺
def transDoToM(doo):
    return doo*101751.561277



In [203]:
import json
with open('D:/Data/JsonData/TainanFood/FinishTainanBigTable.json') as f:
    bigtable=json.load(f)

In [204]:
TainanStationLoc=(22.997183,120.212577)
RedCutBuild=(22.997478,120.202543)
GodRoad=(22.997295,120.197187)

In [205]:
bigtable[0].keys()

dict_keys(['name', 'style', 'Ncomment', 'contentcut', 'wordCount', 'TF_IDF', 'pinCountMaxper5', 'score', 'featurescores', 'tags', 'introduction', 'coordinate', 'opentime', 'closetime', 'address', 'tel'])

In [206]:
bigtable[0]['coordinate'].keys()

dict_keys(['lat', 'lng'])

In [207]:
from collections import Counter

In [208]:
TainanStationDien500={}
for dien in bigtable:
    if dien['style']=="異國料理":
        if dien['coordinate']!=None:
            if (((dien['coordinate']['lat']-TainanStationLoc[0])**2+(dien['coordinate']['lng']-TainanStationLoc[1])**2)**(1/2))*101751.561277<500:
                TainanStationDien500[dien['name']]=dien['score']
print(len(TainanStationDien500))
TainanStationDien500_5=dict(Counter(TainanStationDien500).most_common(5))

68


In [209]:
RedCutBuildDien500={}
for dien in bigtable:
    if dien['style']=="甜點(合併飲品)":
        if dien['coordinate']!=None:
            if (((dien['coordinate']['lat']-RedCutBuild[0])**2+(dien['coordinate']['lng']-RedCutBuild[1])**2)**(1/2))*101751.561277<500:
                RedCutBuildDien500[dien['name']]=dien['score']
print(len(RedCutBuildDien500))
RedCutBuildDien500_5=dict(Counter(RedCutBuildDien500).most_common(5))

55


In [210]:
GodRoadDien500={}
for dien in bigtable:
    if dien['style']=="小吃":
        if dien['coordinate']!=None:
            if (((dien['coordinate']['lat']-GodRoad[0])**2+(dien['coordinate']['lng']-GodRoad[1])**2)**(1/2))*101751.561277<500:
                GodRoadDien500[dien['name']]=dien['score']
print(len(GodRoadDien500))
GodRoadDien500_5=dict(Counter(GodRoadDien500).most_common(5))

185


In [211]:
dienVec={dien['name']:dien['featurescores'] for dien in bigtable}
dienVec

{'同記安平豆花(安平總店)': {'份量大': 0.13,
  '偏甜': 0.24,
  '偏酸': 0.2,
  '其他正面情緒': 0.19,
  '划算': 0.19,
  '口味清淡': 0.07,
  '服務好': -0.15,
  '環境好': 0.14,
  '食物很香': 0.22,
  '食物精緻漂亮': 0.19,
  '食物美味': 0.37},
 '深藍咖啡館': {'份量大': 0.04,
  '偏甜': 0.33,
  '其他正面情緒': 0.21,
  '划算': -0.8,
  '小份量': 0.05,
  '服務好': 0.13,
  '環境好': 0.25,
  '重口味': 0.06,
  '食物很香': 0.29,
  '食物精緻漂亮': 0.3,
  '食物美味': 0.61},
 '阿堂鹹粥': {'份量大': 0.48,
  '偏甜': 0.12,
  '偏辣': 0.02,
  '其他正面情緒': 0.24,
  '划算': -0.23,
  '口味清淡': 0.02,
  '服務好': 0.09,
  '環境好': 0.04,
  '重口味': 0.08,
  '食物很香': 0.1,
  '食物精緻漂亮': 0.23,
  '食物美味': 0.61},
 '富盛號碗粿': {'份量大': 0.4,
  '偏甜': 0.41,
  '偏辣': 0.09,
  '其他正面情緒': 0.24,
  '划算': 0.18,
  '口味清淡': 0.04,
  '小份量': 0.03,
  '服務好': 0.11,
  '環境好': 0.29,
  '重口味': 0.14,
  '食物很香': 0.12,
  '食物精緻漂亮': 0.24,
  '食物美味': 0.58},
 '依蕾特布丁奶酪(安平門市)': {'份量大': 0.08,
  '偏甜': 0.29,
  '偏酸': 0.2,
  '其他正面情緒': 0.15,
  '划算': 0.06,
  '口味清淡': 0.07,
  '小份量': 0.05,
  '環境好': 0.08,
  '食物很香': 0.29,
  '食物精緻漂亮': 0.1,
  '食物美味': 0.55},
 '食下有約 想法廚房': {'份量大': 0.54,
  '偏甜': 0.28

In [212]:
def like(wordict,word1,word2):
    publicwords=set(wordict[word1])&set(wordict[word2])
    if publicwords==set():
        return {word2:0}
    n=0
    for word in wordict[word1]:
        n+=wordict[word1][word]**2
    m=0
    for word in wordict[word2]:
        m+=wordict[word2][word]**2
    l=0
    for word in publicwords:
        l+=wordict[word1][word]*wordict[word2][word]

    if ((n**(1/2))*(m**(1/2)))==0:
        return {word2:0}
    sim=l/((n**(1/2))*(m**(1/2)))
    
    return {word2:sim}

In [213]:
def checksim(word1,wordict,N=5):
    simi={}
    from collections import Counter
    for word2 in wordict:
        simi.update(like(wordict,word1,word2))
    del simi[word1]
    simi=Counter(simi).most_common(N)
#     print(word1+' 相近詞')
    return dict(simi)

In [214]:
import pprint
dienVec={dien['name']:dien['featurescores'] for dien in bigtable if dien['style']=='甜點(合併飲品)'
         or dien['name'] in TainanStationDien500_5}
TainanStation=[{'一推':dien,'關聯推':list(checksim(dien,dienVec,N=5).keys())} for dien in TainanStationDien500_5]

In [215]:
print('台南火車站[異國>甜點]')
pprint.pprint(TainanStation)

台南火車站[異國>甜點]
[{'一推': '靖波門港式茶餐廳',
  '關聯推': ['雞蛋發高燒', '金華61鴉片綠豆蒜', 'Perfume Dance 跳舞香水(南紡門店)', '小丞事', '起士公爵']},
 {'一推': '鳳舞呈翔義式漢堡焗烤料理屋',
  '關聯推': ['TASTy 西堤牛排(民族店)', '龍興冰品店', '阿川粉圓', '舒芙里法式烘培坊', '莉莉水果店']},
 {'一推': 'TASTy 西堤牛排(民族店)',
  '關聯推': ['綣綣屋 霜淇淋專賣', '阿川粉圓', '太陽牌冰品', '鳳舞呈翔義式漢堡焗烤料理屋', '舒芙里法式烘培坊']},
 {'一推': '哞王 原味炭烤牛排',
  '關聯推': ['貝伊拉手工鮮奶酪(永福門市)', '佳里御品紅豆', '冰鄉', '東東鮮蝦餅', '中一豆花']},
 {'一推': 'The Warehouse Steakhouse 食倉鮮切牛排',
  '關聯推': ['好禮道商行', 'Churro & Sweet', '呷甜甜', '葡吉麵包店', '銀波布丁(台南安平店)']}]


In [216]:
import pprint

dienVec={dien['name']:dien['featurescores'] for dien in bigtable if dien['style']=='異國料理' or dien['style']=='鍋類'
         or dien['name'] in RedCutBuildDien500_5}
RedCutBuild=[{'一推':dien,'關聯推':list(checksim(dien,dienVec,N=5).keys())} for dien in RedCutBuildDien500_5]

In [217]:
print('赤崁樓[甜點飲料>主餐]')
pprint.pprint(RedCutBuild)

赤崁樓[甜點飲料>主餐]
[{'一推': '天使貝可 天然酵母手感烘焙',
  '關聯推': ['小餐桌2F (預約制)',
          '小倆口',
          '菓爾Pasta',
          '甘簞行瓦 x Mvsa 西班牙酒莊餐廳',
          '精彩火鍋(台南德安店)']},
 {'一推': 'Autumn 熱·鬆餅',
  '關聯推': ['帕特里夏義大利麵餐廳', '洛義義式餐廳', 'SCK 賽真預約制廚房', '良物肉鋪子', '大大茶樓(台南德安店)']},
 {'一推': 'ぺコぺコ PEKO PEKO',
  '關聯推': ['瑪哈印度餐廳', 'Autumn舒芙蕾熱鬆餅', '晶英軒', '小金魚泰式小館', '漢來海港自助餐廳']},
 {'一推': '人人好冰菓室',
  '關聯推': ['石二鍋(台南中華東店)', '捌伍貳冰室', '桂河泰式料理', '波雷克堤', '新迦拿鍋燒專賣店']},
 {'一推': '奇奇亞義式手工冰淇淋',
  '關聯推': ['麻花重慶火鍋',
          "Perfetto Caff'e完美咖啡",
          '饗食天堂(台南小西門店)',
          '金福氣南洋食堂-台南東寧店',
          '真心食府']}]


In [218]:
import pprint
dienVec={dien['name']:dien['featurescores'] for dien in bigtable  if dien['style']=='飲品'
         or dien['name'] in GodRoadDien500_5}
GodRoad=[{'一推':dien,'關聯推':list(checksim(dien,dienVec,N=5).keys())} for dien in GodRoadDien500_5]

In [219]:
print('神農路海安路[小吃>飲品]')
pprint.pprint(GodRoad)

神農路海安路[小吃>飲品]
[{'一推': '一緒燒日式串燒居酒屋',
  '關聯推': ['席瑪朵咖啡烘培棧',
          '1982 Life House',
          '地球咖啡烘焙美食(永康旗艦店)',
          '艾斯柏泡泡冰',
          'N23度工夫茶']},
 {'一推': '湯勺 Soup Ladle',
  '關聯推': ['老胡記', '江水號', '阿卿傳統飲品‧冰品', '戀戀芒果冰', 'OILILY CAFE']},
 {'一推': '台南味葉家小卷米粉(二店)',
  '關聯推': ['Starbucks Coffee 統一星巴克(長榮門市)',
          '友愛青草茶',
          '泡泡桔子(善化店)',
          '奇異果水果店',
          'SMILE']},
 {'一推': '曉璘海產店',
  '關聯推': ['HORM CAFE&TEA  晃咖啡',
          '蘋果森林休閒餐飲',
          '義豐冬瓜茶(青年店)',
          '荷田美食生活館',
          'Starbucks Coffee 統一星巴克(東山門市)']},
 {'一推': '永上海產碳烤店',
  '關聯推': ['茶桶複合式餐飲',
          '鼓豆咖啡 Great Bean Company',
          '椿之味薏仁湯(永康店)',
          '紅番茄生活飲食屋',
          '茶香世家']}]
